In [94]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim library
from modsim import *

# set the random number generator
np.random.seed(7)
import math

In [95]:
limeshare = State(number_of_scooters = 20, av_batt_life = 100, size_of_region = 2, send_truck = -1)
limeshare

,values
number_of_scooters,20
av_batt_life,100
size_of_region,2
send_truck,-1


In [96]:
def step(state, hour):
#     find number of users per hour
    demand = calcDemand(hour)
    
#     find the av time of use per rider from number of users found above
    length_of_use = calcLengthUsed(hour)
    
#     calc change in battery 
    calcBattDelta(state, demand, length_of_use)
    
#     find hour to send truck
    if(state.av_batt_life < 20 and state.send_truck == -1):
        sendTruck(state, hour)


In [97]:
def calcDemand(hour):
#     based on time of hour with higher prob in noon 4 traffic rates with sets time of day with max number of scooters
    return 5

def calcLengthUsed(hour):
#     based on time used realtive to region, hour, demand with max av battry life 
    return 10
        
def calcBattDelta(state, demand, length_of_use):
    s = state.number_of_scooters 
    c = state.av_batt_life
    t = demand
    
    state.av_batt_life =  math.floor(((s-t)*c + t*(c- length_of_use)) / s)
        

def sendTruck(state, hour):
    state.send_truck = hour;

In [98]:
def run_simulation(state, num_steps):
    
    results = TimeSeries()
    for i in range(num_steps):
        step(state, i + 1)
        results[i] = state.av_batt_life

In [99]:
limeshare = State(number_of_scooters = 20, av_batt_life = 80, size_of_region = 3, send_truck = -1)
run_simulation(limeshare, 24)
limeshare

,values
number_of_scooters,20
av_batt_life,8
size_of_region,3
send_truck,21
